In [1]:
import pandas as pd
from neo4j import GraphDatabase
import requests
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j', 'neo4j'))

Get artworks information

In [4]:
artwork_info = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t')

In [5]:
artwork_info['Artist Info URL'] = artwork_info['Artist Info URL'].progress_apply(lambda x: x if 'andrea-del-verrocchio' not in x\
                                                                  else '/'.join(x.split('/')[:-1] + ['andrea-del-vorrochio']))
artwork_info

100%|██████████| 4119/4119 [00:00<00:00, 263922.06it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


In [6]:
def mod_painting_info(link):
    l = link.split('/')
    l[-2] = 'andrea-del-verrochio'
    return '/'.join(l)

In [7]:
artwork_info['Painting Info URL'] = artwork_info['Painting Info URL'].progress_apply(lambda x: x if 'andrea-del-verrocchio' not in x\
                                                                  else mod_painting_info(x))
artwork_info

100%|██████████| 4119/4119 [00:00<00:00, 267605.42it/s]


,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin


In [8]:
artwork_info['Artist Info URL'] = artwork_info['Artist Info URL'].map(lambda x: f"{x}?json=2")
artwork_info

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring?json=2
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai?...
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller?json=2
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie?json=2
...,...,...,...,...,...,...,...,...
4114,577287aeedc2cb388007fba9,Color Field Painting,Rupprecht Geiger,OE 260,1957,https://uploads7.wikiart.org/images/rupprecht-...,https://www.wikiart.org/en/rupprecht-geiger/oe...,https://www.wikiart.org/en/rupprecht-geiger?js...
4115,57728001edc2cb3880efddcf,Surrealism,Oscar Dominguez,M&#225;quina de coser electro-sexual,1934,https://uploads4.wikiart.org/images/oscar-domi...,https://www.wikiart.org/en/oscar-dominguez/m-q...,https://www.wikiart.org/en/oscar-dominguez?json=2
4116,57728ac7edc2cb3880123cc2,Neo-Expressionism,Georg Baselitz,Female Nude on a Kitchen Chair,1979,https://uploads2.wikiart.org/images/georg-base...,https://www.wikiart.org/en/georg-baselitz/fema...,https://www.wikiart.org/en/georg-baselitz?json=2
4117,57728412edc2cb3880fc9ff4,Expressionism,Marie Laurencin,Apollinaire and His Friends,1909,https://uploads0.wikiart.org/images/marie-laur...,https://www.wikiart.org/en/marie-laurencin/apo...,https://www.wikiart.org/en/marie-laurencin?json=2


In [9]:
def get_painting(id: str):
    base = 'https://www.wikiart.org/en/api/2/Painting?id={id}'
    return requests.get(base.format(id=id)).json()

In [10]:
artwork_info['api_v2'] = artwork_info.ID.progress_apply(lambda x: 'title' in get_painting(x))
artwork_info

 19%|█▊        | 767/4119 [16:33<1:12:22,  1.30s/it]

KeyboardInterrupt



In [ ]:
artwork_info.ID.iloc[500:600].progress_apply(lambda x: 'title' in get_painting(x))